In [116]:
import pandas as pd
from AUIDFunctions import *
import inspect
uh = pd.read_csv("input/unit-history_demo.csv")
am = pd.read_csv("input/all-members_demo.csv")
ac = pd.read_csv("input/accused_demo.csv", dtype={"Middle.Initial":str})
am = am.rename(columns = {'TID' : 'ID1'})
uh = uh.rename(columns = {'TID' : 'ID2'})

In [118]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [119]:
TakeFirst4 = lambda x: x[:4]
BY_to_CA = lambda x: 2016 - x
def AddColumns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "CA_BY"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(TakeFirst4)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(TakeFirst4)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "CA_BY" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df



In [120]:
amd = am.dropna(axis=1, how='all')
uhd = uh.dropna(axis=1, how='all')
amdm = AddColumns(amd)
uhdm = AddColumns(uhd)
print(amd.columns)
print(uhd.columns)
cols = [col for col in amd.columns if col in uhd.columns]
print(cols)

Index(['ID1', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Gender', 'Race', 'Birth.Year', 'F4FN', 'F4LN',
       'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
Index(['ID2', 'First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name',
       'Appointed.Date', 'Current.Age', 'Gender', 'Race', 'Star1', 'Star2',
       'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8', 'Star9', 'Star10',
       'Current.Unit', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1'],
      dtype='object')
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'F4FN', 'F4LN', 'Current.Age.p1', 'Current.Age.m1']


In [121]:
mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t1 = amdmp.merge(uhdmp, on = mc, how ='inner')

mc = ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"]
amdmp = RemoveDuplicates(amdm)
uhdmp = RemoveDuplicates(uhdm)
t2 = amdmp.merge(uhdmp, on = mc, how ='inner')

print(intersect(t1['ID2'], t2['ID2']))
print(intersect(t1['ID1'], t2['ID1']))
print(t1.shape)
print(t2.shape)

[]
[]
(1312, 29)
(4289, 29)


In [122]:
def LoopMerge(df1, df2, on_list, keep_columns = ['ID1', 'ID2']):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    # Generate on list?
    for mc in on_list:
        df1t = RemoveDuplicates(df1[keep_columns[:1] + mc], mc)
        df2t = RemoveDuplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    return (dfm.reset_index(drop=True), df1, df2)



In [123]:
m = LoopMerge(amdm, uhdm, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', 'Last.Name',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Suffix.Name", "Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN',"Middle.Initial",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Middle.Initial",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.m1"],
    ['First.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race", "Current.Age.p1"]
])

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
4289
******
******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
1312
******
******
['F4FN', 'F4LN', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
19949
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.p1']
6011
******
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Age.m1']
3
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Middle.Initial', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
******
['First.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Dat

In [124]:
uid_ref = pd.concat([m[0][["ID1", "ID2"]], m[1][["ID1"]], m[2][["ID2"]]]).reset_index(drop=True)
if "UID" not in uid_ref.columns:
    uid_ref.insert(0, "UID", uid_ref.index + 1)
am.head()

,ID1,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Gender,Race,Birth.Year
0,1,JEFFERY,AARON,M,NaN,2005-09-26,MALE,WHITE,1971
1,2,KARINA,AARON,NaN,NaN,2005-09-26,FEMALE,HISPANIC,1980
2,3,DANIEL,ABATE,P,NaN,1970-06-15,MALE,WHITE,1942
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,MALE,WHITE,1968
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,MALE,WHITE,1942


In [125]:
uid_ref.head()

,UID,ID1,ID2
0,1,2,2
1,2,16,16
2,3,17,17
3,4,32,31
4,5,33,32


In [126]:
am = am.merge(uid_ref[["ID1", "UID"]], on='ID1', how='left')
uh = uh.merge(uid_ref[["ID2", "UID"]], on='ID2', how='left')

In [128]:
ref = pd.concat([am, uh])
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star2,Star3,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605


In [129]:
ref['Middle.Initial'] = ref['Middle.Initial'].fillna("")
print(ref.columns)
ref.to_csv("test-reference.csv", index=False)

Index(['Appointed.Date', 'Birth.Year', 'Current.Age', 'Current.Unit',
       'First.Name', 'Gender', 'ID1', 'ID2', 'Last.Name', 'Middle.Initial',
       'Race', 'Star1', 'Star10', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6',
       'Star7', 'Star8', 'Star9', 'Suffix.Name', 'UID'],
      dtype='object')


In [104]:
ref.head()

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID,UID_x,UID_y
0,2005-09-26,1971.0,NaN,NaN,JEFFERY,MALE,1,NaN,AARON,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5603,5603,5603
1,2005-09-26,1980.0,NaN,NaN,KARINA,FEMALE,2,NaN,AARON,,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,1,1
2,1970-06-15,1942.0,NaN,NaN,DANIEL,MALE,3,NaN,ABATE,P,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25552,25552,25552
3,1994-12-05,1968.0,NaN,NaN,ANTHONY,MALE,4,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5604,5604,5604
4,1969-01-06,1942.0,NaN,NaN,CARMEL,MALE,5,NaN,ABBATE,G,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5605,5605,5605


In [102]:
ref

,Appointed.Date,Birth.Year,Current.Age,Current.Unit,First.Name,Gender,ID1,ID2,Last.Name,Middle.Initial,...,Star4,Star5,Star6,Star7,Star8,Star9,Suffix.Name,UID,UID_x,UID_y
31592,1999-04-01,NaN,45.0,19.0,PETER,MALE,NaN,31593,ZYGOWICZ,,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,25547,25547,25547
31593,1986-10-13,NaN,59.0,650.0,DEBRA,FEMALE,NaN,31594,ZYLVITIS,,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,25548,25548,25548
31594,2002-12-02,NaN,41.0,14.0,MARK,MALE,NaN,31595,ZYMANTAS,,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,25549,25549,25549
31595,1996-12-02,NaN,45.0,20.0,CARLO,MALE,NaN,31596,ZYRKOWSKI,,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,25550,25550,25550
31596,1955-02-08,NaN,83.0,631.0,STANLEY,MALE,NaN,31597,ZYSKOWSKI,,...,-999.0,-999.0,-999.0,-999.0,-999.0,-999.0,NaN,25551,25551,25551


In [57]:
'''profiles = AggregateData(ref, uid = "UID", id_cols = [],
              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'''

'profiles = AggregateData(ref, uid = "UID", id_cols = [],\n              max_cols = setdiff(ref.columns, ["UID"]), make_uid=False)'

In [58]:
uid = "UID"
uid_col = [uid]
agg_df = ref[uid_col].drop_duplicates().reset_index(drop=True) 
max_cols = setdiff(ref.columns, ["UID"])
print(max_cols)

['Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2', 'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit', 'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender', 'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10']


In [59]:
profiles = ref[uid_col + max_cols].drop_duplicates()
profiles = profiles.groupby(uid_col, as_index=False)[max_cols]

In [60]:
profiles = profiles.agg(max)

In [61]:
profiles.columns

Index(['UID', 'Appointed.Date', 'ID2', 'Star7', 'Star4', 'Star9', 'Star2',
       'Star5', 'First.Name', 'Last.Name', 'Star1', 'Current.Unit',
       'Current.Age', 'ID1', 'Suffix.Name', 'Star3', 'Gender',
       'Middle.Initial', 'Star8', 'Birth.Year', 'Star6', 'Race', 'Star10'],
      dtype='object')

In [64]:
acd = ac.dropna(axis=1, how='all')
acd = AddColumns(acd)
profiles = AddColumns(profiles)

In [75]:
m = LoopMerge(profiles, acd, on_list = [
    ['First.Name', 'Last.Name', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Suffix.Name",'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', "Middle.Initial", 'Appointed.Date', 'Gender', "Race"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race", "Current.Unit"],
    ['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', "Race"],
    ['F4FN', 'F4LN', 'Appointed.Date', 'Gender', "Race"],
    
], keep_columns = ["UID", "TID"])
for i in m:
    print(i.shape)

******
['First.Name', 'Last.Name', 'Middle.Initial', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
2
******
******
['First.Name', 'Last.Name', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
13381
******
******
['F4FN', 'F4LN', 'Suffix.Name', 'Appointed.Date', 'Gender', 'Race']
1
******
******
['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', 'Race', 'Current.Unit']
15
******
******
['First.Name', 'Last.Name', 'Appointed.Date', 'Gender', 'Race']
1
******
(13400, 3)
(18543, 27)
(2622, 16)
